# Механизми на вниманието и трансформери

Един от основните недостатъци на рекурентните мрежи е, че всички думи в последователността имат еднакво влияние върху резултата. Това води до неефективност при стандартните LSTM encoder-decoder модели за задачи от тип последователност към последователност, като разпознаване на именувани обекти и машинен превод. В действителност, определени думи в входната последователност често имат по-голямо влияние върху изходните последователности от други.

Да разгледаме модел от тип последователност към последователност, като машинен превод. Той се реализира чрез две рекурентни мрежи, където едната мрежа (**encoder**) компресира входната последователност в скрито състояние, а другата (**decoder**) разгъва това скрито състояние в преведен резултат. Проблемът с този подход е, че крайната състояние на мрежата трудно запомня началото на изречението, което води до ниско качество на модела при дълги изречения.

**Механизмите на вниманието** предоставят начин за претегляне на контекстуалното влияние на всеки входен вектор върху всяка изходна прогноза на RNN. Това се реализира чрез създаване на преки връзки между междинните състояния на входната RNN и изходната RNN. По този начин, при генериране на изходен символ $y_t$, ще вземем предвид всички входни скрити състояния $h_i$, с различни теглови коефициенти $\alpha_{t,i}$.

![Изображение, показващо модел encoder/decoder с добавен слой за внимание](../../../../../translated_images/bg/encoder-decoder-attention.7a726296894fb567.webp)
*Моделът encoder-decoder с механизъм за добавено внимание в [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf), цитиран от [този блог пост](https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html)*

Матрицата на вниманието $\{\alpha_{i,j}\}$ представлява степента, до която определени входни думи участват в генерирането на дадена дума в изходната последователност. По-долу е даден пример за такава матрица:

![Изображение, показващо пример за подравняване, намерено от RNNsearch-50, взето от Bahdanau - arviz.org](../../../../../translated_images/bg/bahdanau-fig3.09ba2d37f202a6af.webp)

*Фигура, взета от [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf) (Фиг.3)*

Механизмите на вниманието са отговорни за голяма част от настоящите или почти настоящите върхови постижения в обработката на естествен език. Добавянето на внимание обаче значително увеличава броя на параметрите на модела, което води до проблеми със скалирането при RNN. Основно ограничение при скалирането на RNN е, че рекурентната природа на моделите затруднява групирането и паралелизирането на обучението. В RNN всеки елемент от последователността трябва да бъде обработен в последователен ред, което означава, че не може лесно да бъде паралелизиран.

Приемането на механизми на вниманието в комбинация с това ограничение доведе до създаването на съвременните трансформерни модели, които познаваме и използваме днес, като BERT и OpenGPT3.

## Трансформерни модели

Вместо да предават контекста на всяка предишна прогноза в следващата стъпка на оценка, **трансформерните модели** използват **позиционни кодировки** и внимание, за да уловят контекста на даден вход в предоставен прозорец от текст. Изображението по-долу показва как позиционните кодировки с внимание могат да уловят контекста в даден прозорец.

![Анимиран GIF, показващ как се извършват оценките в трансформерните модели.](../../../../../lessons/5-NLP/18-Transformers/images/transformer-animated-explanation.gif)

Тъй като всяка входна позиция се съпоставя независимо с всяка изходна позиция, трансформерите могат да се паралелизират по-добре от RNN, което позволява създаването на много по-големи и изразителни езикови модели. Всяка глава на вниманието може да се използва за изучаване на различни взаимоотношения между думите, което подобрява задачите за обработка на естествен език.

**BERT** (Bidirectional Encoder Representations from Transformers) е много голяма многослойна трансформерна мрежа с 12 слоя за *BERT-base* и 24 за *BERT-large*. Моделът първо се предварително обучава върху голям корпус от текстови данни (Wikipedia + книги) с помощта на неконтролирано обучение (предсказване на маскирани думи в изречение). По време на предварителното обучение моделът усвоява значително ниво на езиково разбиране, което след това може да бъде използвано с други набори от данни чрез фина настройка. Този процес се нарича **трансферно обучение**.

![изображение от http://jalammar.github.io/illustrated-bert/](../../../../../translated_images/bg/jalammarBERT-language-modeling-masked-lm.34f113ea5fec4362.webp)

Съществуват много вариации на трансформерните архитектури, включително BERT, DistilBERT, BigBird, OpenGPT3 и други, които могат да бъдат фино настроени. Пакетът [HuggingFace](https://github.com/huggingface/) предоставя хранилище за обучение на много от тези архитектури с PyTorch.

## Използване на BERT за текстова класификация

Нека видим как можем да използваме предварително обучен модел BERT за решаване на нашата традиционна задача: класификация на последователности. Ще класифицираме оригиналния ни набор от данни AG News.

Първо, нека заредим библиотеката HuggingFace и нашия набор от данни:


In [10]:
import torch
import torchtext
from torchnlp import *
import transformers
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_len = len(vocab)

Loading dataset...
Building vocab...


Тъй като ще използваме предварително обучен BERT модел, ще трябва да използваме специфичен токенизатор. Първо, ще заредим токенизатор, свързан с предварително обучен BERT модел.

Библиотеката HuggingFace съдържа хранилище с предварително обучени модели, които можете да използвате, като просто посочите техните имена като аргументи към функциите `from_pretrained`. Всички необходими бинарни файлове за модела ще бъдат автоматично изтеглени.

Въпреки това, понякога може да се наложи да заредите свои собствени модели. В този случай можете да посочите директорията, която съдържа всички съответни файлове, включително параметрите за токенизатора, файла `config.json` с параметрите на модела, бинарните тегла и др.


In [11]:
# To load the model from Internet repository using model name. 
# Use this if you are running from your own copy of the notebooks
bert_model = 'bert-base-uncased' 

# To load the model from the directory on disk. Use this for Microsoft Learn module, because we have
# prepared all required files for you.
bert_model = './bert'

tokenizer = transformers.BertTokenizer.from_pretrained(bert_model)

MAX_SEQ_LEN = 128
PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)

Обектът `tokenizer` съдържа функцията `encode`, която може да се използва директно за кодиране на текст:


In [15]:
tokenizer.encode('PyTorch is a great framework for NLP')

[101, 1052, 22123, 2953, 2818, 2003, 1037, 2307, 7705, 2005, 17953, 2361, 102]

Тогава нека създадем итератори, които ще използваме по време на обучението за достъп до данните. Тъй като BERT използва своя собствена функция за кодиране, ще трябва да дефинираме функция за запълване, подобна на `padify`, която сме дефинирали преди:


In [4]:
def pad_bert(b):
    # b is the list of tuples of length batch_size
    #   - first element of a tuple = label, 
    #   - second = feature (text sequence)
    # build vectorized sequence
    v = [tokenizer.encode(x[1]) for x in b]
    # compute max length of a sequence in this minibatch
    l = max(map(len,v))
    return ( # tuple of two tensors - labels and features
        torch.LongTensor([t[0] for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v])
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=8, collate_fn=pad_bert, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=8, collate_fn=pad_bert)

В нашия случай ще използваме предварително обучен BERT модел, наречен `bert-base-uncased`. Нека заредим модела, използвайки пакета `BertForSequenceClassfication`. Това гарантира, че нашият модел вече има необходимата архитектура за класификация, включително финалния класификатор. Ще видите предупредително съобщение, което посочва, че теглата на финалния класификатор не са инициализирани и моделът ще изисква предварително обучение - това е напълно нормално, защото точно това ще направим!


In [9]:
model = transformers.BertForSequenceClassification.from_pretrained(bert_model,num_labels=4).to(device)

Some weights of the model checkpoint at ./bert were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ./bert and

Сега сме готови да започнем обучението! Тъй като BERT вече е предварително обучен, искаме да започнем с доста малка скорост на обучение, за да не разрушим началните тегла.

Цялата тежка работа се извършва от модела `BertForSequenceClassification`. Когато извикаме модела върху тренировъчните данни, той връща както загубата, така и изхода на мрежата за входния минибатч. Използваме загубата за оптимизация на параметрите (`loss.backward()` извършва обратното преминаване), а `out` за изчисляване на точността на обучението, като сравняваме получените етикети `labs` (изчислени с помощта на `argmax`) с очакваните `labels`.

За да контролираме процеса, натрупваме загубата и точността през няколко итерации и ги отпечатваме на всеки `report_freq` цикли на обучение.

Това обучение вероятно ще отнеме доста време, затова ограничаваме броя на итерациите.


In [6]:
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

report_freq = 50
iterations = 500 # make this larger to train for longer time!

model.train()

i,c = 0,0
acc_loss = 0
acc_acc = 0

for labels,texts in train_loader:
    labels = labels.to(device)-1 # get labels in the range 0-3         
    texts = texts.to(device)
    loss, out = model(texts, labels=labels)[:2]
    labs = out.argmax(dim=1)
    acc = torch.mean((labs==labels).type(torch.float32))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    acc_loss += loss
    acc_acc += acc
    i+=1
    c+=1
    if i%report_freq==0:
        print(f"Loss = {acc_loss.item()/c}, Accuracy = {acc_acc.item()/c}")
        c = 0
        acc_loss = 0
        acc_acc = 0
    iterations-=1
    if not iterations:
        break

Loss = 1.1254194641113282, Accuracy = 0.585
Loss = 0.6194715118408203, Accuracy = 0.83
Loss = 0.46665248870849607, Accuracy = 0.8475
Loss = 0.4309701919555664, Accuracy = 0.8575
Loss = 0.35427074432373046, Accuracy = 0.8825
Loss = 0.3306886291503906, Accuracy = 0.8975
Loss = 0.30340143203735354, Accuracy = 0.8975
Loss = 0.26139299392700194, Accuracy = 0.915
Loss = 0.26708646774291994, Accuracy = 0.9225
Loss = 0.3667240524291992, Accuracy = 0.8675


Можете да видите (особено ако увеличите броя на итерациите и изчакате достатъчно дълго), че класификацията с BERT ни дава доста добра точност! Това е така, защото BERT вече разбира доста добре структурата на езика и ние само трябва да дообучим крайния класификатор. Въпреки това, тъй като BERT е голям модел, целият процес на обучение отнема много време и изисква сериозна изчислителна мощност! (GPU, и за предпочитане повече от един).

> **Note:** В нашия пример използвахме един от най-малките предварително обучени модели на BERT. Съществуват по-големи модели, които вероятно ще дадат по-добри резултати.


## Оценка на производителността на модела

Сега можем да оценим производителността на нашия модел върху тестовия набор от данни. Цикълът за оценка е доста подобен на цикъла за обучение, но не трябва да забравяме да превключим модела в режим на оценка, като извикаме `model.eval()`.


In [10]:
model.eval()
iterations = 100
acc = 0
i = 0
for labels,texts in test_loader:
    labels = labels.to(device)-1      
    texts = texts.to(device)
    _, out = model(texts, labels=labels)[:2]
    labs = out.argmax(dim=1)
    acc += torch.mean((labs==labels).type(torch.float32))
    i+=1
    if i>iterations: break
        
print(f"Final accuracy: {acc.item()/i}")

Final accuracy: 0.9047029702970297


## Основни моменти

В този модул видяхме колко лесно е да вземем предварително обучен езиков модел от библиотеката **transformers** и да го адаптираме към нашата задача за класификация на текст. По същия начин, моделите BERT могат да се използват за извличане на обекти, отговаряне на въпроси и други задачи, свързани с обработка на естествен език.

Моделите на трансформъри представляват най-съвременните технологии в NLP и в повечето случаи трябва да бъдат първото решение, с което започвате експерименти, когато внедрявате персонализирани NLP решения. Въпреки това, разбирането на основните принципи на рекурентните невронни мрежи, обсъдени в този модул, е изключително важно, ако искате да изграждате усъвършенствани невронни модели.



---

**Отказ от отговорност**:  
Този документ е преведен с помощта на AI услуга за превод [Co-op Translator](https://github.com/Azure/co-op-translator). Въпреки че се стремим към точност, моля, имайте предвид, че автоматизираните преводи може да съдържат грешки или неточности. Оригиналният документ на неговия роден език трябва да се счита за авторитетен източник. За критична информация се препоръчва професионален човешки превод. Ние не носим отговорност за каквито и да е недоразумения или погрешни интерпретации, произтичащи от използването на този превод.
